In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

tf.get_logger().setLevel('ERROR')

# read the csv file
CLT_df = pd.read_csv('collecting data.csv', encoding='cp1252')


# Custom RRMSE loss function
def rrmse(y_true, y_pred):
    numerator = tf.sqrt(tf.reduce_mean(tf.square((y_true - y_pred) / y_true)))
    return numerator

# Load the saved model with custom_objects argument
ANN_model_A33 = load_model('ANN_model_A33.h5', custom_objects={'rrmse': rrmse})
ANN_model_D33 = load_model('ANN_model_D33.h5', custom_objects={'rrmse': rrmse})
ANN_model_f11 = load_model('ANN_model_f11.h5', custom_objects={'rrmse': rrmse})
ANN_model_f16 = load_model('ANN_model_f16.h5', custom_objects={'rrmse': rrmse})
ANN_model_f33 = load_model('ANN_model_f33.h5', custom_objects={'rrmse': rrmse})
ANN_model_f35 = load_model('ANN_model_f35.h5', custom_objects={'rrmse': rrmse})
ANN_model_f55 = load_model('ANN_model_f55.h5', custom_objects={'rrmse': rrmse})
ANN_model_f66 = load_model('ANN_model_f66.h5', custom_objects={'rrmse': rrmse})

# Function to standardize the input values
def standardize_input(value, mean, std):
    return (value - mean) / std

# Function for model prediction using a custom function
def predict_with_model(model, input_data):
    return model.predict(input_data)

# Calculate the mean and standard deviation for each input feature from the training data
w_mean = CLT_df['w'].mean()
w_std = CLT_df['w'].std()
h1_mean = CLT_df['h1'].mean()
h1_std = CLT_df['h1'].std()
h2_mean = CLT_df['h2'].mean()
h2_std = CLT_df['h2'].std()
h3_mean = CLT_df['h3'].mean()
h3_std = CLT_df['h3'].std()
h4_mean = CLT_df['h4'].mean()
h4_std = CLT_df['h4'].std()
s_mean = CLT_df['s'].mean()
s_std = CLT_df['s'].std()
E1_mean = CLT_df['E1'].mean()
E1_std = CLT_df['E1'].std()
G12_mean = CLT_df['G12'].mean()
G12_std = CLT_df['G12'].std()
G23_mean = CLT_df['G23'].mean()
G23_std = CLT_df['G23'].std()

# Prompt the user to enter the input values
w = float(input("Enter the width 'mm': "))
h1 = float(input("Enter the thickness of the first layer 'mm': "))
h2 = float(input("Enter the thickness of the second layer 'mm': "))
h3 = float(input("Enter the thickness of the third layer 'mm': "))
h4 = float(input("Enter the thickness of the fourth layer 'mm': "))
s = float(input("Enter the spacing 'mm': "))
E1 = float(input("Enter the longitudinal Young's modulus 'MPa': "))
G12 = float(input("Enter the longitudinal shear modulus 'MPa': "))
G23 = float(input("Enter the rolling shear modulus 'MPa': "))

# Standardize the input values
scaled_w = standardize_input(w, w_mean, w_std)
scaled_h1 = standardize_input(h1, h1_mean, h1_std)
scaled_h2 = standardize_input(h2, h2_mean, h2_std)
scaled_h3 = standardize_input(h3, h3_mean, h3_std)
scaled_h4 = standardize_input(h4, h4_mean, h4_std)
scaled_s = standardize_input(s, s_mean, s_std)
scaled_E1 = standardize_input(E1, E1_mean, E1_std)
scaled_G12 = standardize_input(G12, G12_mean, G12_std)
scaled_G23 = standardize_input(G23, G23_mean, G23_std)

# Convert the standardized input values to a NumPy array
input_data = np.array([[scaled_w, scaled_h1, scaled_h2, scaled_h3, scaled_h4, scaled_s, scaled_E1, scaled_G12, scaled_G23]])

# Make predictions using the custom prediction function
A33_predicted_value = predict_with_model(ANN_model_A33, input_data)
D33_predicted_value = predict_with_model(ANN_model_D33, input_data)
f11_predicted_value = predict_with_model(ANN_model_f11, input_data)
f16_predicted_value = predict_with_model(ANN_model_f16, input_data)
f33_predicted_value = predict_with_model(ANN_model_f33, input_data)
f35_predicted_value = predict_with_model(ANN_model_f35, input_data)
f55_predicted_value = predict_with_model(ANN_model_f55, input_data)
f66_predicted_value = predict_with_model(ANN_model_f66, input_data)

# In-plane shear stiffness closed-form solution calculation
N = None  # Number of layers
h = None  # Thickness
b = w + s # width of unit-cell

# Check conditions and calculate N and h
if h3 == 0:
    N = 3
    h = (2 * h1 + h2) / 3
    # Calculate f_22(RM)^(3-ply)
    f_22RM_closed_form = (6 / 5) * b / (w * h * G12)
    
elif h4 == 0 and h3 != 0:
    N = 5
    h = (2 * h1 + 2 * h2 + h3) / 5
    # Calculate alpha_3_s
    S3 = (5/6) * w * h
    I_n = (h ** 3) * w / 12
    alpha_3_s = s / (G12 * S3) + (s ** 3) / (12 * E1* I_n)

    # Calculate f_22(RM)
    f_22RM_closed_form = (2 * alpha_3_s) / (N - 1) + (b**2 * (N**2 - 2*N - 3)) / (2 * h * w**2 * (N**2 - 2*N - 2)**2) * \
         (((b - 2*w) * (N**2 - 2*N - 2) - b)**2 / (b**2 * G12 * (N**2 - 1) * (N - 3)) + \
          (6 / (5 * G23)) * ((N**2 - 2*N + 5) / (N - 1)))
    
else:
    N = 7
    h = (2 * h1 + 2 * h2 + 2 * h3 + h4) / 7
    # Calculate alpha_3_s
    S3 = (5/6) * w * h
    I_n = (h ** 3) * w / 12
    alpha_3_s = s / (G12 * S3) + (s ** 3) / (12 * E1* I_n)

    # Calculate f_22(RM)
    f_22RM_closed_form = (2 * alpha_3_s) / (N - 1) + (b**2 * (N**2 - 2*N - 3)) / (2 * h * w**2 * (N**2 - 2*N - 2)**2) * \
         (((b - 2*w) * (N**2 - 2*N - 2) - b)**2 / (b**2 * G12 * (N**2 - 1) * (N - 3)) + \
          (6 / (5 * G23)) * ((N**2 - 2*N + 5) / (N - 1)))

# Calculate K_Theta
K_Theta = (w ** 4 * (G12 + G23)) / (6 * h)

# Calculate alpha_n_star
alpha_n_star = ((24 * b - 19 * w) / (20 * G12 * w * h)) + (((b - w) ** 3) / (E1 * w ** 3 * h))

# Calculate J
J = (w * h ** 3) / 16 * (16/3 - 3.36 * (h/w) * (1 - (1/12) * (h/w) ** 4))

# Calculate alpha_3_s
S3 = (5/6) * w * h
I_n = (h ** 3) * w / 12
alpha_3_s = s / (G12 * S3) + (s ** 3) / (12 * E1* I_n)


# Calculate A_33
A_33_closed_form = (N - 1) / ((b ** 2 / (2 * K_Theta)) + (2 * N * alpha_n_star / (N + 1)))

# Calculate D_33
D_33_closed_form = (N * G12 * J) / (2 * b) + (N * h ** 2 * (N - 1)) / 6 * ((b ** 2 / (K_Theta * (N - 2)))
                                                                           + (4 * alpha_n_star / (N + 1))) ** -1

# Calculate f_11(RM)
f_11RM_closed_form = (2 * alpha_3_s) / (N + 1) + (b ** 2 * (N ** 2 + 2 * N - 3)) / (2 * h * w ** 2 * (N ** 2 + 2 * N - 2) ** 2) * \
                (((b - 2 * w) * (N ** 2 + 2 * N - 2) - b) ** 2 / (b ** 2 * G12 * (N - 1) ** 2 * (N + 3)) + \
                (6 / (5 * G23)) * ((N ** 2 + 2 * N + 5) / (N + 1)))


# Calculate the in-plane shear stiffness
A_33 = A33_predicted_value * A_33_closed_form
in_plane_stiffness = A_33[0][0]  # Extract the actual value from the NumPy array

# Calculate the torsional stiffness
D_33 = D33_predicted_value * D_33_closed_form
torsional_stiffness = D_33[0][0]  # Extract the actual value from the NumPy array

# Calculate the shear compliance f11
f_11 = f11_predicted_value * f_11RM_closed_form
f_11 = f_11[0][0]  # Extract the actual value from the NumPy array

# Calculate the shear compliance f16
f_16 = f16_predicted_value * f_11RM_closed_form
f_16 = f_16[0][0]  # Extract the actual value from the NumPy array

# Calculate the shear compliance f33
f_33 = f33_predicted_value * f_22RM_closed_form
f_33 = f_33[0][0]  # Extract the actual value from the NumPy array

# Calculate the shear compliance f35
f_35 = f35_predicted_value * f_22RM_closed_form
f_35 = f_35[0][0]  # Extract the actual value from the NumPy array

# Calculate the shear compliance f55
f_55 = f55_predicted_value * f_22RM_closed_form
f_55 = f_55[0][0]  # Extract the actual value from the NumPy array

# Calculate the shear compliance f66
f_66 = f66_predicted_value * f_11RM_closed_form
f_66 = f_66[0][0]  # Extract the actual value from the NumPy array

# Format values using scientific notation
def format_scientific(value):
    coefficient, exponent = f"{value:.2e}".split("e")
    coefficient = float(coefficient)
    return f"{coefficient:.2f} × 10^{int(exponent)}"

# Print the results
print("\n------- Input Values -------")
print(f"Width: {w} mm")
print(f"Thickness of the first layer: {h1} mm")
print(f"Thickness of the second layer: {h2} mm")
print(f"Thickness of the third layer: {h3} mm")
print(f"Thickness of the fourth layer: {h4} mm")
print(f"Spacing: {s} mm")
print(f"Longitudinal Young's modulus: {E1} N/mm^2")
print(f"Longitudinal shear modulus: {G12} N/mm^2")
print(f"Rolling shear modulus: {G23} N/mm^2")

print("\n------- Calculated Results -------")
print(f"A_33: {in_plane_stiffness:.2f} N/mm")
print(f"D_33: {torsional_stiffness:.2f} N.mm")
print(f"f_11: {format_scientific(f_11)} mm/N")
print(f"f_16: {format_scientific(f_16)} mm/N")
print(f"f_33: {format_scientific(f_33)} mm/N")
print(f"f_35: {format_scientific(f_35)} mm/N")
print(f"f_55: {format_scientific(f_55)} mm/N")
print(f"f_66: {format_scientific(f_66)} mm/N")
# Calculate f11_RM, f22_RM, Delta (RM/BG)
import math
# Calculate Delta (RM-BG)
Delta_numerator = (
    abs(-(5/9) * f_11 + (4/9) * math.sqrt(2) * f_16 + (2/9) * f_66) ** 2 +
    abs((2/9) * f_11 + (2/9) * math.sqrt(2) * f_16 - (8/9) * f_66) ** 2 +
    abs(-(8/9) * f_33 + (2/9) * math.sqrt(2) * f_35 + (2/9) * f_55) ** 2 +
    abs((2/9) * f_33 + (4/9) * math.sqrt(2) * f_35 - (5/9) * f_55) ** 2 +
    2 * abs((2/9) * math.sqrt(2) * f_11 - (5/9) * f_16 + (1/9) * math.sqrt(2) * f_66) ** 2 +
    2 * abs((1/9) * math.sqrt(2) * f_33 - (5/9) * f_35 + (2/9) * math.sqrt(2) * f_55) ** 2
)

Delta_denominator = math.sqrt(
    abs(f_11) ** 2 + 2 * abs(f_16) ** 2 + abs(f_33) ** 2 + 2 * abs(f_35) ** 2 + abs(f_55) ** 2 + abs(f_66) ** 2
)

Delta = math.sqrt(Delta_numerator) / Delta_denominator

f11_RM = (4/9) * f_11 + (4/9) * (2 ** 0.5) * f_16 + (2/9) * f_66
f22_RM = (4/9) * f_55 + (4/9) * (2 ** 0.5) * f_35 + (2/9) * f_33
# Print the value of Delta, f11_RM and f22_RM
print("Delta (RM,BG) =", "{:.4f}".format(Delta))
print(f"f11_RM: {format_scientific(f11_RM)} mm/N")
print(f"f22_RM: {format_scientific(f22_RM)} mm/N")


Enter the width 'mm': 120
Enter the thickness of the first layer 'mm': 20
Enter the thickness of the second layer 'mm': 20
Enter the thickness of the third layer 'mm': 0
Enter the thickness of the fourth layer 'mm': 0
Enter the spacing 'mm': 50
Enter the longitudinal Young's modulus 'MPa': 8000
Enter the longitudinal shear modulus 'MPa': 400
Enter the rolling shear modulus 'MPa': 100
1/1 [==============================] - 0s 130ms/step

------- Input Values -------
Width: 120.0 mm
Thickness of the first layer: 20.0 mm
Thickness of the second layer: 20.0 mm
Thickness of the third layer: 0.0 mm
Thickness of the fourth layer: 0.0 mm
Spacing: 50.0 mm
Longitudinal Young's modulus: 8000.0 N/mm^2
Longitudinal shear modulus: 400.0 N/mm^2
Rolling shear modulus: 100.0 N/mm^2

------- Calculated Results -------
A_33: 15148.78 N/mm
D_33: 5491841.00 N.mm
f_11: 2.57 × 10^-4 mm/N
f_16: 1.86 × 10^-4 mm/N
f_33: 3.18 × 10^-4 mm/N
f_35: 1.49 × 10^-4 mm/N
f_55: 1.79 × 10^-4 mm/N
f_66: 1.35 × 10^-4 mm/N
De